In [1]:
import torch
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import dialogflow

In [3]:
import re

In [4]:
model = AutoModelForCausalLM.from_pretrained("gpt2-sv")
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
with open('starters.txt', 'r', encoding = 'utf-8') as f:
    starters = f.read().strip().split('\n')
starters

['когда приходит понедельник',
 'cперва жена моя сбежала',
 'я женщинам не доверяю',
 'в наш век высоких технологий',
 'с утра немного поработал',
 'у губернатора дилемма',
 'аркадий не любил пробежки',
 'татарский хан сурово крикнул',
 'плохой купил ты телевизор',
 'сломаться может всё на свете']

In [6]:
def get_answer(prefix):
    end_token_id = tokenizer.encode('***')[0]
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        end_token=end_token_id,
        do_sample=False, 
        max_length=size+60, 
        repetition_penalty=10., 
        temperature=1,
        num_beams=10,
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    result1 = prefix + result.split('\n')[0]
    result2 = result.split('\n')[2]

    return '\n\n***BONUS***\n'.join([result1, result2])

In [7]:
print(get_answer(starters[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


когда приходит понедельник я открываю холодильник и вижу там говна столько что мне бы хватило до середины февраля

***BONUS***
олег в костюме супермэна пришёл на первое свиданье с оксаной а у неё план по спасению утопающих мужыков


### bot

In [8]:
!set GOOGLE_APPLICATION_CREDENTIALS = 'D:/GeekBrains/nlp/les15/ai-nlp-bot-9nju.json'

In [9]:
DIALOGFLOW_PROJECT_ID = 'ai-nlp-bot-9nju' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'ai_oleg_story_bot'  # ID бота из телеграма
TOKEN = "1950225126:AAEpyzTBshSCeWJRBNYq3DKEAbRhFy2BmGA"

In [10]:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'D:/GeekBrains/nlp/les15/ai-nlp-bot-9nju.json'# скачнный JSON
updater = Updater(TOKEN, use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher

In [11]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('какой стишок ты хочешь?')

    
def textMessage(update: Update, context: CallbackContext):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    
    text_input = update.message.text
    if text_input == 'start':
        update.message.reply_text('скажи первую строчку')
    else:
        update.message.reply_text(get_answer(text_input))

In [12]:
dispatcher.add_handler(CommandHandler('start', startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))
# Начинаем поиск обновлений
updater.start_polling()
updater.idle()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
